# Random-to-Random

Deck with $n$ cards.

**Random-to-Random Shuffle**:
 1. Remove a card u.a.r. from the deck
 1. Insert it u.a.r.

One shuffle acts on the deck by one of the following elements of $S_n$:
$$S=\{(1)\}\cup\{(a\ a+1)\ :\ a\in [n-1]\}\cup\{(a\ a+1\ \cdots b)^\pm\ :\ a, b\in [n], |a-b| > 1\} \subset S_n.$$

Note: $|S|=1+(n-1)+2n(n-3)$ ?

For any $x\in S_n$, and $s\in S$, $\mathbb{P}(x, xs)=\begin{cases}\frac{1}{n} & s=(1)\\ \frac{2}{n^2} & s= (a\ a+1)\\ \frac{1}{n^2}&s=(a\ a+1\ \cdots\ b)\end{cases}$

**Modified Random-to-Random Shuffle**
 1. Remove a card u.a.r. from the deck
 1. Insert it u.a.r. *among all positions, excluding the one below it or the top position if the bottom card is selected*
 
For any $x\in S_n$, $\mathbb{P}(x, xs)=\begin{cases}\frac{1}{n} & s=(1)\\ \frac{1-\frac{1}{n}}{|S|-1} & s\in S\setminus\{(1)\}\end{cases}$

**Goal**

Let $T$ be the first time that all cards have been picked ($T\geq n$).

We want to compute $\mathbb{P}(X_t=\sigma\mid T\leq t)$ for all $\sigma\in S_n$ and $t$.

___

In [37]:
import random as r
import itertools as it

In [3]:
def get_deck(n):
    return list(range(1, n + 1))

def rtr(deck:list):
    n = len(deck)
    card = r.choice(deck)
    new_deck = deck.copy()
    new_deck.remove(card)
    new_loc = r.randint(0, n - 1)
    new_deck.insert(b, card)
    return new_deck, card

def m_rtr(deck:list):
    n = len(deck)
    a = r.randint(0, n - 1)
    card = deck[a]
    new_deck = deck.copy()
    new_deck.remove(card)
    b = r.choice([pos % n for pos in range(a + 2, a + n + 1)])
    new_deck.insert(b, card)
    return new_deck, card

In [4]:
def T_trial(shuffle, n):
    deck = get_deck(n)
    marked = set()
    t = 0
    while len(marked) < n:
        deck, card = shuffle(deck)
        marked.add(card)
        t += 1
    return t

____

In [5]:
class Node:
    def __init__(self, deck:list, marked:set=set(), depth:int=0):
        self.deck = deck
        self.marked = marked
        self.children = []
        self.depth = depth # number of shuffles that have been made
        
    def set_deck(self, deck):
        self.deck = deck
    
    def set_marked(self, marked):
        self.marked = marked
        
    def iterate_children(self):
        for child in self.children:
            yield child
        return None
    
    def __str__(self, indent=0):
        return (" " * (indent * 6)) + ("\n" + (" " * (indent * 6))).join([
            "_" * (len(self.deck)*2 + 1),
            "|" + "|".join([" ", "*"][card in self.marked] for card in self.deck) + "|" + f"   depth: {self.depth}",
            "|" + "|".join(str(card) for card in self.deck) + "|",
            "‾" * (len(self.deck)*2 + 1),
        ])
    
    def __repr__(self):
        return str(self)
    
    def generate_children(self):
        """
        make every possible move
        create a new child node for each move
        link in children list
        """
        
        n = len(self.deck)
        
        for position, card in enumerate(self.deck):
            for pre_new_loc in range(position + 2, position + 1 + n):
                new_deck = self.deck.copy()
                new_deck.remove(card)
                new_loc = pre_new_loc % n
                new_deck.insert(new_loc, card)
                new_marked = self.marked.copy()
                new_marked.add(card)
                node = Node(new_deck, new_marked, self.depth + 1)
                self.children.append(node)

In [90]:
class Tree:
    def __init__(self, n):
        self.n = n
        self.top = Node(get_deck(n))
    
    def __str__(self):
        ret = ""
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            ret += node.__str__(node.depth) + "\n"
            for child in node.children:
                stack.append(child)
        return ret
    
    def __repr__(self):
        return str(self)
    
    def expand(self, max_depth:int):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth >= max_depth:
                continue
            if node.children == []:
                node.generate_children()
            for child in node.children:
                stack.append(child)
        
    def get_leaves(self, depth:int, full:bool=False):
        stack = [self.top]
        while len(stack) > 0:
            node = stack.pop()
            if node.depth == depth:
                if not full or len(node.marked) == self.n:
                    yield node
            else:
                for child in node.children:
                    stack.append(child)
        return None
    
    def count_leaves(self, depth:int):
        
        self.expand(depth)
        
        data = {str(list(deck)):0 for deck in it.permutations(range(1, self.n + 1))}
        for node in self.get_leaves(depth, full=True):
            data[str(node.deck)] += 1
        for deck in data:
            print(deck, " ", data[deck])
        return None
        
    
    def P(self, t:int, sigma:list):
        if t < self.n:
            raise BaseException(f"t must be at least n={self.n}")
        if sorted(sigma) != get_deck(self.n):
            raise BaseException("sigma must be a permutation of n cards")
        
        self.expand(t)
        
        count = 0
        total = 0
        for node in self.get_leaves(t, full=True):
            total += 1
            if node.deck == sigma:
                count += 1
        return count / total
    
    
        

___

In [106]:
tree = Tree(3)

In [107]:
tree.count_leaves(3)

[1, 2, 3]   13
[1, 3, 2]   7
[2, 1, 3]   7
[2, 3, 1]   12
[3, 1, 2]   3
[3, 2, 1]   6


In [108]:
tree

_______
| | | |   depth: 0
|1|2|3|
‾‾‾‾‾‾‾
      _______
      | | |*|   depth: 1
      |1|2|3|
      ‾‾‾‾‾‾‾
            _______
            | | |*|   depth: 2
            |1|2|3|
            ‾‾‾‾‾‾‾
                  _______
                  | | |*|   depth: 3
                  |1|2|3|
                  ‾‾‾‾‾‾‾
                  _______
                  | |*| |   depth: 3
                  |1|3|2|
                  ‾‾‾‾‾‾‾
                  _______
                  | |*|*|   depth: 3
                  |1|2|3|
                  ‾‾‾‾‾‾‾
                  _______
                  |*| |*|   depth: 3
                  |2|1|3|
                  ‾‾‾‾‾‾‾
                  _______
                  |*| |*|   depth: 3
                  |1|2|3|
                  ‾‾‾‾‾‾‾
                  _______
                  | |*|*|   depth: 3
                  |2|3|1|
                  ‾‾‾‾‾‾‾
            _______
            | |*| |   depth: 2
            |1|3|2|
            ‾‾‾‾‾‾‾
                  